<a href="https://colab.research.google.com/github/scotthou94/applied_deep_learning/blob/master/hw3_p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hw3-P1
The dataset used in this part is created based on the loader notebook provided by instructor. Each class has 1k images and we have 10 classes in total. The dataset is saved to my google drive and made public. 

In [0]:
from __future__ import print_function

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import functools, itertools, json, os, re, textwrap
import tensorflow as tf
tf.enable_eager_execution()
tfe = tf.contrib.eager
from sklearn.model_selection import train_test_split

from PIL import Image, ImageDraw
from six.moves.urllib import request
from xml.dom import minidom

In [0]:
# Get the dataset from google drive

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1SSIO4PbdlvpyTYMxvBn0phY8ba3DgLRX'
downloaded = drive.CreateFile({'id': file_id})
# Save to local file
target_path = 'quickdraw.npz'
downloaded.GetContentFile(target_path)

In [4]:
# Inspect the data
with open(target_path, 'r') as f:
  loaded = np.load(f)
  X, y = loaded['X'], loaded['y']
# Reshape the data so it has 1 color channel
num = X.shape[0]
a = X.shape[1]
X = X.reshape((num, a, a, 1))
print(X.shape, y.shape)

(10000, 64, 64, 1) (10000,)


In [5]:
# See the data scale
print(X[0].shape, y[0])

(64, 64, 1) zebra


### Steps
1. Data preparation
1. Split into train/validation/test
1. Build model
1. Compile
1. Train
1. Evaluate on test set

In [0]:
# Data normalization
X = X.astype('float32') / 255.0

In [7]:
# Turn the y label to numeric format
zoo = ['elephant', 'giraffe', 'kangaroo', 'lion', 'monkey', 'panda',
       'penguin', 'rhinoceros', 'tiger', 'zebra']
num = range(10)
mapping = {}
for animal, d in zip(zoo, num):
  mapping[animal] = d
  print(animal, d)
for i in range(len(y)):
  y[i] = mapping[y[i]]

elephant 0
giraffe 1
kangaroo 2
lion 3
monkey 4
panda 5
penguin 6
rhinoceros 7
tiger 8
zebra 9


In [8]:
# Split into train/validation/test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, shuffle=True)

# Convert label to one-hot encoding
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

print("Train size", x_train.shape, y_train.shape)
print("Validation size", x_val.shape, y_val.shape)
print("Test size", x_test.shape, y_test.shape)

Train size (6750, 64, 64, 1) (6750, 10)
Validation size (2250, 64, 64, 1) (2250, 10)
Test size (1000, 64, 64, 1) (1000, 10)


## Experiment setup
1. Compare batch size: 256, 32. Model 2 and 4
1. Compare learning rate: 0.01/0.001/0.0001. Model 1 - 3
1. Change filter size: (25, 25), (5, 5). Model 4 and 5

### Model 1
1. Learning rate 0.01
1. Batch size 256
1. filter size (5, 5)

In [10]:
# Model
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(64, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(64,64,1))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(10, activation='softmax'))
# Take a look at the model summary
model.summary()

# Compile model
adam1 = tf.train.AdamOptimizer(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
             optimizer=adam1,
             metrics=['accuracy'])
# Train model
batch_size = 256
epochs = 10
model.fit(x_train,
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val, y_val))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 21, 21, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 21, 21, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
__________

### Model 2
1. Learning rates of 0.001
1. Batch size 256
1. Filter size (5, 5)

In [11]:
# Model
model2 = tf.keras.Sequential()

model2.add(tf.keras.layers.Conv2D(64, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(64,64,1))) 
model2.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
model2.add(tf.keras.layers.Dropout(0.3))

model2.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model2.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
model2.add(tf.keras.layers.Dropout(0.3))

model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(256, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.5))

model2.add(tf.keras.layers.Dense(10, activation='softmax'))
# Take a look at the model summary
model2.summary()

# Compile model
lr = 0.001
AdamOptimizer = tf.train.AdamOptimizer(learning_rate=lr)
model2.compile(loss='categorical_crossentropy',
             optimizer=AdamOptimizer,
             metrics=['accuracy'])
# Train model
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                               min_delta=0.01,
                                               patience=0,
                                               verbose=1)
batch_size = 256
epochs = 10
model2.fit(x_train,
           y_train,
           batch_size=batch_size,
           epochs=epochs,
           validation_data=(x_val, y_val))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 64, 64, 64)        1664      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 21, 21, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 21, 21, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)              0         
__________

### Model3
1. Adam optimizer,  learning rate 0.0001
1. Batch size 256
1. Filter size (5, 5)

In [13]:
# Model3
model3 = tf.keras.Sequential()

model3.add(tf.keras.layers.Conv2D(64, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(64,64,1))) 
model3.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
model3.add(tf.keras.layers.Dropout(0.3))

model3.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model3.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
model3.add(tf.keras.layers.Dropout(0.3))

model3.add(tf.keras.layers.Flatten())
model3.add(tf.keras.layers.Dense(256, activation='relu'))
model3.add(tf.keras.layers.Dropout(0.5))

model3.add(tf.keras.layers.Dense(10, activation='softmax'))
# Take a look at the model summary
model3.summary()

# Compile model
Adam = tf.train.AdamOptimizer(learning_rate=0.0001)
model3.compile(loss='categorical_crossentropy',
               optimizer=Adam,
               metrics=['accuracy'])
# Train model
batch_size = 256
epochs = 10
model3.fit(x_train,
           y_train,
           batch_size=batch_size,
           epochs=epochs,
           validation_data=(x_val, y_val))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 64, 64, 64)        1664      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 21, 21, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 21, 21, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 21, 21, 32)        18464     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1568)              0         
__________

### Model4
1. 0.001 learning rate
1. 32 batch size
1. 5 * 5 filter size

In [10]:
# Model4
model4 = tf.keras.Sequential()

model4.add(tf.keras.layers.Conv2D(64, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(64,64,1))) 
model4.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
model4.add(tf.keras.layers.Dropout(0.3))

model4.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model4.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
model4.add(tf.keras.layers.Dropout(0.3))

model4.add(tf.keras.layers.Flatten())
model4.add(tf.keras.layers.Dense(256, activation='relu'))
model4.add(tf.keras.layers.Dropout(0.5))

model4.add(tf.keras.layers.Dense(10, activation='softmax'))
# Take a look at the model summary
model4.summary()

# Compile model
Adam4 = tf.train.AdamOptimizer(learning_rate=0.001)
model4.compile(loss='categorical_crossentropy',
               optimizer=Adam4,
               metrics=['accuracy'])
# Train model
batch_size = 32
epochs = 10
model4.fit(x_train,
           y_train,
           batch_size=batch_size,
           epochs=epochs,
           validation_data=(x_val, y_val))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 64, 64, 64)        1664      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 21, 21, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 21, 21, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)              0         
__________

### Model 5
1. 25 \* 25 and 20 \* 20 filter size
1. 0.001 learning rate
1. Batch size 32

In [9]:
# Model5
model5 = tf.keras.Sequential()

model5.add(tf.keras.layers.Conv2D(64, kernel_size=(25, 25), padding='same', activation='relu', input_shape=(64,64,1))) 
model5.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
model5.add(tf.keras.layers.Dropout(0.3))

model5.add(tf.keras.layers.Conv2D(32, kernel_size=(20, 20), padding='same', activation='relu'))
model5.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
model5.add(tf.keras.layers.Dropout(0.3))

model5.add(tf.keras.layers.Flatten())
model5.add(tf.keras.layers.Dense(256, activation='relu'))
model5.add(tf.keras.layers.Dropout(0.5))

model5.add(tf.keras.layers.Dense(10, activation='softmax'))
# Take a look at the model summary
model5.summary()

# Compile model
Adam5 = tf.train.AdamOptimizer(learning_rate=0.001)
model5.compile(loss='categorical_crossentropy',
               optimizer=Adam5,
               metrics=['accuracy'])
# Train model
batch_size = 32
epochs = 10
model5.fit(x_train,
           y_train,
           batch_size=batch_size,
           epochs=epochs,
           validation_data=(x_val, y_val))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        40064     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 21, 21, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 21, 21, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 32)        819232    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
__________

In [22]:
# Evaluate model 4
test_loss, test_acc = model4.evaluate(x_test, y_test)
print('test loss is {0}, test accuracy is {1}'.format(test_loss, test_acc))

32/32 [==============================] - 0s 15ms/step
test loss is 0.509465979576, test accuracy is 0.812


## Conclusion
1. By comparing model 1 - 3, the best learning rate is 0.001
1. By comparing model 2 and 4, the smaller batch size gives better validation accuracy but not by much
1. By changing the filter size, model 5 gives worse performance than model 4 but still not by much.
1. Here I evaluate the model 4, it gives a test accuracy of 0.812
1. I think there's still room for improvement using fine-tuning but given the purpose of this assignment I think this is a reasonable accuracy.